In [1]:
import json
import numpy as np

import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('sentiwordnet')
nltk.download('omw-1.4')

from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from afinn import Afinn
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import custom_lexicons.senticnet.senticnet as sentic

import numpy as np 
import pandas as pd
import data_reader
import results_analyser
import itertools

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\phoec\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\phoec\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     C:\Users\phoec\AppData\Roaming\nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\phoec\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## Symbolic

In [2]:
FILE_DIR = "./custom_lexicons/"

NTUSD_FILEPATH = "ntusd/NTUSD_Fin_word_v1.0.json"
STOCKTWITLEXI_FILEPATH = "stocktwitlexi/domain_lexicon_raw_norm.csv"
SENTI_DD_FILEPATH = "sentidd/Senti-DD_data1.csv"
LM_FILEPATH = "sentidd/LM_Word_List.csv"

# AFINN
afinn = Afinn()

# VADER
analyzer = SentimentIntensityAnalyzer()

# NTUSD-FIN
with open(FILE_DIR+NTUSD_FILEPATH, "r") as f:
    data = f.read()
    NTUSD = json.loads(data)
word_sent_dict = {}
for i in range(len(NTUSD)):
    word_sent_dict[NTUSD[i]["token"]] = NTUSD[i]["market_sentiment"]
    
# STOCKTWITLEXI    
stocktwitlexi = pd.read_csv(FILE_DIR+STOCKTWITLEXI_FILEPATH, header=None, index_col=0)
stocktwitlexi = stocktwitlexi.to_dict()[1]

# SENTIDD
sentidd = pd.read_csv(FILE_DIR+SENTI_DD_FILEPATH)
sentidd_dict = dict(zip(zip(sentidd.entity, sentidd.directional_word), sentidd.sentiment))
lm_df = pd.read_csv(FILE_DIR+LM_FILEPATH)
lm_dict = dict(zip(lm_df.word, lm_df.label))

stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def remove_stopwords(data):
    sentence_token = [s.split(' ') for s in data] 
    return sentence_token

def get_wordnet_tag(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

In [3]:
def normalise_scores(score, count):
    if (count>0):
        return score/count
    else:
        return score

def standardise_scores(pred_raw, lexicon):
    if(lexicon=="senticnet"): #-1 to 1
        return pred_raw
    elif(lexicon=="ntusd"): #-3.81 to 1.22, range is 5 so /2.5
        return [pred/2.5 for pred in pred_raw]
    elif(lexicon=="sentiwordnet"): #-1 to 1 since pos-neg
        return pred_raw
    elif(lexicon=="stocktwitlexi"): #-1 to 1
        return pred_raw
    elif(lexicon=="afinn"): #-5 to 5
        return [pred/5 for pred in pred_raw]
    elif(lexicon=="vader"): #-1 to 1
        return pred_raw
    elif(lexicon=="sentidd"): #-2 to 2
        return [pred/2 for pred in pred_raw]

In [4]:
def senti_dd_polarity(text, sentidd_dict, lm_dict):
    def lm_score(text, lm_dict):
        tokens = word_tokenize(text)
        count = 0
        score = 0
        for token in tokens:
            try:
                if lm_dict[token]=="positive":
                    score += 1
                    count += 1
                elif lm_dict[token]=="negative":
                    score -= 1
                    count += 1
            except:
                pass

        return score/count if count>0 else score

    def senti_dd_score(text, sentidd_dict):
        tokens = word_tokenize(text)
        count = 0
        score = 0
        stemmed_tokens = [stemmer.stem(token) for token in tokens]
        lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
        for stemmed_token in stemmed_tokens:
            for lemmatized_token in lemmatized_tokens:
                
                try:
                    if (sentidd_dict[(lemmatized_token, stemmed_token)] =="positive"):
                        score += 1
                        count += 1
                    elif (sentidd_dict[(lemmatized_token, stemmed_token)] =="negative"):
                        score -= 1
                        count += 1
                except:
                    pass
        return score/count if count>0 else score
    
    score = lm_score(text, lm_dict)
    context_sentiment_score = senti_dd_score(text, sentidd_dict)
    if context_sentiment_score > 0: score += 1
    elif context_sentiment_score < 0: score -= 1

    return score

In [5]:
def individual_scoring(X, y_class):
    ntusd_pred_raw = []
    stocktwitlexi_pred_raw = []
    afinn_pred_raw = []
    vader_pred_raw = []
    sentidd_pred_raw = []
    
    for test_tweet in X:
        ntusd_score = 0
        stocktwitlexi_score = 0
        afinn_score = 0
        
        ntusd_count = 0
        stocktwitlexi_count = 0
        afinn_count = 0
        
        sentence_tagged = np.array(nltk.pos_tag(test_tweet))
        for tagged in sentence_tagged:
            word = tagged[0]
            wn_tag = get_wordnet_tag(tagged[1])
            
            #NTUSD
            try: 
                ntusd_score += word_sent_dict[word]
                ntusd_count += 1
            except:
                pass

            #Stocktwitlexi
            try: 
                stocktwitlexi_score += stocktwitlexi[word]
                stocktwitlexi_count += 1
            except:
                pass
            
            #Afinn
            try: 
                afinn_score += afinn.score(word)
                afinn_count += 1
            except:
                pass
        
        s = " ".join(test_tweet)
        
        #Vader
        vader_output = analyzer.polarity_scores(s)
        vader_score = vader_output["compound"] 
        
        #Senti-DD
        sentidd_score = senti_dd_polarity(s, sentidd_dict, lm_dict)
        
        ntusd_score = normalise_scores(ntusd_score, ntusd_count)
        stocktwitlexi_score = normalise_scores(stocktwitlexi_score, stocktwitlexi_count)
        afinn_score = normalise_scores(afinn_score, afinn_count)
        vader_score = vader_score #already normalised
        sentidd_score = sentidd_score #already normalised
        
        ntusd_pred_raw.append(ntusd_score)
        stocktwitlexi_pred_raw.append(stocktwitlexi_score)
        afinn_pred_raw.append(afinn_score)
        vader_pred_raw.append(vader_score)
        sentidd_pred_raw.append(sentidd_score)
        
    ntusd_pred = standardise_scores(ntusd_pred_raw, "ntusd")
    stocktwitlexi_pred = standardise_scores(stocktwitlexi_pred_raw, "stocktwitlexi")
    afinn_pred = standardise_scores(afinn_pred_raw, "afinn")
    vader_pred = standardise_scores(vader_pred_raw, "vader")
    sentidd_pred = standardise_scores(sentidd_pred_raw, "sentidd")

    combined_df = pd.DataFrame()
    combined_df['ntusd'] = ntusd_pred
    combined_df['afinn'] = afinn_pred
    combined_df['vader'] = vader_pred
    combined_df['stocktwitlexi'] = stocktwitlexi_pred
    combined_df['sentidd'] = sentidd_pred
    combined_df['actual_class'] = y_class
    
    return combined_df

In [6]:
def combine_voting_leave2soft(row):
    lowest_col = row.sort_values().idxmin()
    highest_col = row.sort_values().idxmax()

    return row.sort_values().iloc[1:4].mean()

## Subsymbolic

In [7]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup

# check if it is running with GPU or not
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

cuda:0


In [8]:
# helper function to load the data
def data_handler(root, train_filename, dev_filename):
    train_data_raw = pd.read_csv(root + train_filename)
    val_data_raw = pd.read_csv(root + dev_filename)
    X_train, y_train = train_data_raw['text_cleaned'], train_data_raw['Label']
    X_val, y_val = val_data_raw['text_cleaned'], val_data_raw['Label']
    # concatenate the train and validation set text_cleaned data to get the token ids for the words
    frames = [X_train, X_val]
    X = pd.concat(frames)
    # convert from pandas dataframe to numpy arrays
    X_ = X.to_numpy()
    return X_train, y_train, X_val, y_val, X_

# Create a function to tokenize a set of texts
def preprocessing_for_bert(data):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (np.array): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model.
    """
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in data:
        # `encode_plus` will:
        #    (1) Tokenize the sentence
        #    (2) Add the `[CLS]` and `[SEP]` token to the start and end
        #    (3) Truncate/Pad sentence to max length
        #    (4) Map tokens to their IDs
        #    (5) Create attention mask
        #    (6) Return a dictionary of outputs
        encoded_sent = tokenizer.encode_plus(
            text=sent,
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=MAX_LEN,             # Max length to truncate/pad
            pad_to_max_length=True,         # Pad sentence to max length
            #return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask=True      # Return attention mask
            )
        
        # Add the outputs to the lists
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

# tokenize data
# Specify `MAX_LEN`
MAX_LEN = 256

def create_data_loader(root,train_filename,dev_filename,batch_size=16):
    X_train, y_train, X_val, y_val, X_ = data_handler(root, train_filename, dev_filename)

    # Concatenate train data and test data
    all_texts = np.concatenate([X_train, X_val])

    # Encode our concatenated data
    encoded_texts = [tokenizer.encode(sent, add_special_tokens=True) for sent in all_texts]

    # Find the maximum length
    max_len = max([len(sent) for sent in encoded_texts])

    # Print sentence 0 and its encoded token ids
    token_ids = list(preprocessing_for_bert([X_[0]])[0].squeeze().numpy())

    # Run function `preprocessing_for_bert` on the train set and the validation set
#     print('Tokenizing data...')
    train_inputs, train_masks = preprocessing_for_bert(X_train)
    val_inputs, val_masks = preprocessing_for_bert(X_val)

    # Convert other data types to torch.Tensor
    train_labels = torch.tensor(y_train)
    val_labels = torch.tensor(y_val)

    # Create the DataLoader for our training set
    train_data = TensorDataset(train_inputs, train_masks, train_labels)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    # Create the DataLoader for our validation set
    val_data = TensorDataset(val_inputs, val_masks, val_labels)
    val_sampler = SequentialSampler(val_data)
    val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

    return train_dataloader, val_dataloader, y_val

In [9]:
%%time
# Create the BertClassfier class
class BertClassifier(nn.Module):
    def __init__(self, freeze_bert=False):
        super(BertClassifier, self).__init__()
        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        D_in, H, D_out = 768, 50, 2

        # Instantiate BERT model
        self.bert = BertModel.from_pretrained('bert-base-uncased')

        # Instantiate an one-layer feed-forward classifier
        self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            #nn.Dropout(0.5),
            nn.Linear(H, D_out)
        )

        # Freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):
        # Feed input to BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)

        return logits
    
def initialize_model(train_dataloader, epochs=4):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    # Instantiate Bert Classifier
    bert_classifier = BertClassifier(freeze_bert=False)

    # Tell PyTorch to run the model on GPU
    bert_classifier.to(device)

    # Create the optimizer
    optimizer = AdamW(bert_classifier.parameters(),
                      lr=5e-5,    # Default learning rate
                      eps=1e-8    # Default epsilon value
                      )

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs

    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

# ROC curve and AUC value to test the value on the validation set
def bert_predict(model, test_dataloader):
    """Perform a forward pass on the trained BERT model to predict probabilities
    on the test set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    all_logits = []

    # For each batch in our test set...
    for batch in test_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask = tuple(t.to(device) for t in batch)[:2]

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)
        all_logits.append(logits)
    
    # Concatenate logits from each batch
    all_logits = torch.cat(all_logits, dim=0)

    # Apply softmax to calculate probabilities
    probs = F.softmax(all_logits, dim=1).cpu().numpy()

    return probs

Wall time: 0 ns


## Generate results

In [10]:
def generate_results(data, model, symbolic_weightage, results_df):
    
    #Symbolic
    if(data=="data1"):
        X, y_class = data_reader.read_data1("list")
    elif(data=="data2"):
        X, y_class = data_reader.read_data2("list")
    elif(data=="data3"):
        X, y_class = data_reader.read_data3("list")
        
    combined_df = individual_scoring(X, y_class)
    combination = ["ntusd", "stocktwitlexi", "afinn", "vader", "sentidd"]
    y_class = combined_df['actual_class']
    curr_df = combined_df[combination]
    voting_leave2soft = curr_df.apply(combine_voting_leave2soft, axis=1)
    voting_leave2soft = voting_leave2soft.to_list()
    
    #Subsymbolic
    batch_size = 16
    test_filepath = data + "_test.csv"
    _, val_loader, y_class = create_data_loader(root=DATA_PATH,
                                          train_filename=test_filepath,
                                          dev_filename=test_filepath,
                                          batch_size=batch_size)
    bert_probs = bert_predict(model, val_loader)
    bert_probs = [(2*prob[1])-1 for prob in bert_probs] #convert to -1 to 1

    #Combine
    overall_probs = [(x*symbolic_weightage)+(y*(1-symbolic_weightage)) for x,y in zip(voting_leave2soft, bert_probs)]
    pred_class = results_analyser.probability_to_class(overall_probs)
    
    return results_analyser.calculate_metrics(results_df, y_class, pred_class, data+"_symbolic"+str(+symbolic_weightage))

In [11]:
cwd = os.getcwd()
DATA_PATH = cwd+"/data/subsymbolic/"
SAVED_MODEL_PATH = cwd+"/model/"

combined_bert_model = "bert_model_combined_data.pt"
combined_train_filepath = "combined_data_train.csv"
combined_val_filepath = "combined_data_val.csv"

batch_size = 16
epo = 5

# load the data
train_loader, val_loader, y_val_data1 = create_data_loader(root=DATA_PATH,
                                                     train_filename=combined_train_filepath,
                                                     dev_filename=combined_val_filepath,
                                                     batch_size=batch_size)

# call the saved weights
PATH = SAVED_MODEL_PATH+combined_bert_model
model, _, _ = initialize_model(train_loader,epochs=epo)
model.load_state_dict(torch.load(PATH))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\phoec\anaconda3\envs\tf2\lib\site-packages\transformers\tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Some weights of the model checkpoint at bert-base-uncased were not used when initializi

<All keys matched successfully>

## Test on Data 2 and 3

In [12]:
results_data2 = pd.DataFrame()
results_data3 = pd.DataFrame()

symbolic_weights = [0.2,0.4,0.6,0.8]
for p in symbolic_weights:
    results_data2 = generate_results("data2", model, p, results_data2)
    
display(results_data2)


for p in symbolic_weights:
    results_data3 = generate_results("data3", model, p, results_data3)
    
display(results_data3)

C:\Users\phoec\anaconda3\envs\tf2\lib\site-packages\transformers\tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
C:\Users\phoec\anaconda3\envs\tf2\lib\site-packages\transformers\tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to p

,Accuracy,Experiment,F1_score,Precision,Recall
0,0.752000,data2_symbolic0.2,0.813734,0.742947,0.899431
1,0.761143,data2_symbolic0.4,0.820908,0.748437,0.908918
2,0.773714,data2_symbolic0.6,0.831058,0.755039,0.924099
3,0.794286,data2_symbolic0.8,0.845626,0.771518,0.935484


C:\Users\phoec\anaconda3\envs\tf2\lib\site-packages\transformers\tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
C:\Users\phoec\anaconda3\envs\tf2\lib\site-packages\transformers\tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to p

,Accuracy,Experiment,F1_score,Precision,Recall
0,0.734711,data3_symbolic0.2,0.792453,0.729529,0.867257
1,0.737295,data3_symbolic0.4,0.794058,0.732254,0.867257
2,0.744186,data3_symbolic0.6,0.799189,0.737828,0.871681
3,0.756245,data3_symbolic0.8,0.807089,0.750317,0.873156


## Save results

In [14]:
RESULTS_FILE_DIR = "./results/"

results_data2.to_csv(RESULTS_FILE_DIR+"result2_df_hybrid.csv", index=False)
results_data3.to_csv(RESULTS_FILE_DIR+"result3_df_hybrid.csv", index=False)